In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import pandas as pd
import time
from langdetect import detect
from pop_artists import top_pop_artists
from utils import clean_lyrics, get_english_lyrics
from tqdm import tqdm


# ------------------------------
# Set up API Credentials
# ------------------------------

SPOTIFY_CLIENT_ID = "1e9f17fbe8f84b3ebabe5d48603e6ffc"
SPOTIFY_CLIENT_SECRET = "6f33b108987544418206cb70fd94d403"
GENIUS_ACCESS_TOKEN = "tRFnVnQJ2Ltli7ANNbqLg3I_Ir2NX7TX_iS6L7ufNGWFldulVJjGOmYiVb5w_EDl"

# ------------------------------
# Authenticate with Spotify & Genius
# ------------------------------

auth_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)
genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN, timeout=15, retries=3)


# ------------------------------
# Data Collection from Spotify & Genius
# ------------------------------

data = []

for artist_name in tqdm(top_pop_artists, desc="Processing artists"):
    artist_results = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
    if not artist_results['artists']['items']:
        print(f"Artist {artist_name} not found on Spotify.")
        continue

    artist = artist_results['artists']['items'][0]
    artist_id = artist['id']
    print(f"Collecting top tracks for {artist_name}...")

    top_tracks = sp.artist_top_tracks(artist_id, country='US')['tracks']

    for track in top_tracks:
        track_title = track['name']
        track_artists = [a['name'] for a in track['artists']]
        is_collaboration = len(track_artists) > 1
        track_id = track['id']
        track_duration = track['duration_ms'] / 1000
        album_info = track['album']
        release_date = album_info.get('release_date', 'Unknown')
        popularity = track.get('popularity', 0)

        lyrics = get_english_lyrics(track_title, artist_name, genius)
        time.sleep(1)

        data.append({
            "song_title": track_title,
            "artists": ", ".join(track_artists),
            "release_date": release_date,
            "popularity": popularity,
            "is_collaboration": is_collaboration,
            "duration_seconds": track_duration,
            "spotify_id": track_id,
            "lyrics": lyrics
        })

df = pd.DataFrame(data)
print("Data collection complete. Here's a preview:")
print(df.head())

Searching for "Cruel Summer" by Taylor Swift...
Done.
Searching for "Fortnight (feat. Post Malone)" by Taylor Swift...
Done.
Non-English lyrics for Fortnight (feat. Post Malone) by Taylor Swift, retrying with 'English' hint...
Searching for "Fortnight (feat. Post Malone) English" by Taylor Swift...
Done.
Searching for "cardigan" by Taylor Swift...
Done.
Searching for "Lover" by Taylor Swift...
Done.
Searching for "august" by Taylor Swift...
Done.
Searching for "I Can Do It With a Broken Heart" by Taylor Swift...
Done.
Searching for "Blank Space" by Taylor Swift...
Done.
Searching for "I Don’t Wanna Live Forever (Fifty Shades Darker)" by Taylor Swift...
No results found for: 'I Don’t Wanna Live Forever (Fifty Shades Darker) Taylor Swift'
Searching for "Don’t Blame Me" by Taylor Swift...
Done.
Searching for "Anti-Hero" by Taylor Swift...
Done.
Searching for "Handlebars (feat. Dua Lipa)" by Dua Lipa...
Done.
Searching for "Cold Heart - PNAU Remix" by Dua Lipa...
Done.
Searching for "Train

In [ ]:
df.to_csv("spotify_genius_data.csv", index=False)